# Using MarkerCount in R

#### You must to install MarkerCount first by running the following command in a terminal
pip install MarkerCount

#### Then Load MarkerCount in R

In [9]:
suppressPackageStartupMessages(library(anndata))
suppressPackageStartupMessages(library(reticulate))

### Load MarkerCount
mkrcnt <- import("MarkerCount.marker_count")

## Reference based MarkerCount
#### Load data

In [10]:
adata_test <- read_h5ad('Melanoma_5K.h5ad')
adata_ref <- read_h5ad('HeadNeck_6K.h5ad')

#### Set data to pass to MarkerCount

In [11]:
X_ref <- adata_ref$to_df()
cell_type_ref <- adata_ref$obs[,'broad_cell_type']

X_test <- adata_test$to_df()
cell_type_test <- adata_test$obs[,'broad_cell_type']

In [12]:
as.character(unique(cell_type_ref))
as.character(unique(cell_type_test))

[1] "Fibroblast"       "Tumor"            "B cell"           "Myocyte"         
 [5] "Unknown"          "Macrophage"       "Endothelial cell" "T cell"          
 [9] "Dendritic cell"   "Mast cell"

[1] "B cell"           "T cell"           "Tumor"            "Unknown"         
[5] "Fibroblast"       "Macrophage"       "Endothelial cell" "NK cell"

#### Run MarkerCount-Ref

In [8]:
undesired <- c('Tumor', 'Unknown')

In [10]:
df_res <- mkrcnt$MarkerCount_Ref( X_ref, cell_type_ref, 
                                  X_test = X_test, df_mkr_mat = NULL,
                                  N_mkrs = 18, of_th = 0.9, # min_th = 0.2, 
                                  cell_types_to_excl = undesired, 
                                  cluster_label = NULL, X_pca = NULL, 
                                  log_transformed = FALSE, 
                                  file_to_save_marker = 'UC_cell_markers', 
                                  verbose = TRUE)

df_res[,'cell_type_org'] <- cell_type_test
head(df_res)

,cluster_label,cell_type_pred,cell_type_org
,<chr>,<chr>,<fct>
Cy72_CD45_H02_S758_comb,7,B cell,B cell
CY58_1_CD45_B02_S974_comb,22,T cell,T cell
Cy71_CD45_D08_S524_comb,20,unassigned,Tumor
Cy81_FNA_CD45_B01_S301_comb,10,Fibroblast,Tumor
Cy80_II_CD45_B07_S883_comb,11,Fibroblast,Tumor
Cy81_Bulk_CD45_B10_S118_comb,10,B cell,Tumor


## Marker-based MarkerCount
#### Load data

In [8]:
adata_test <- read_h5ad('Melanoma_5K.h5ad')
adata_test

AnnData object with n_obs × n_vars = 4513 × 23684
    obs: 'tumor', 'malignant', 'cell_type', 'broad_cell_type', 'sample_name'
    var: 'gene_name'

#### Set data to pass to MarkerCount

In [ ]:
X_test <- adata_test$to_df()
cell_type_test <- adata_test$obs[,'broad_cell_type']

#### Load marker matrix

In [6]:
df_mkr_mat <- read.csv('Mkr_mat_CellMarker_Blood.csv', header = TRUE, row.names = 1)
head(df_mkr_mat)

,X11.SEP,ABCA2,ABCA7,ABCC3,ABHD15,ABHD3,ABI2,ABI3,ABLIM1,ACOT9,⋯,ZFAT,ZFYVE26,ZNF101,ZNF366,ZNF506,ZNF527,ZNF600,ZNF683,ZNF789,ZNFX1
,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,⋯,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
Effector CD8+ memory T (Tem) cell,0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,1,0,0,0
Fibroblast,0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
Naive T cell,0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
Regulatory T (Treg) cell,0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
CD4+ memory T cell,0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
Megakaryocyte,0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0


#### Run MarkerCount 

In [7]:
df_res <- mkrcnt$MarkerCount( X_test, mkr_mat = df_mkr_mat,
                              min_th = 0.2, N_mkrs_max = 18, N_mkrs_min = 3, of_th = 0.9,
                              target_FPR = 0.12, clustering = 'gm', N_clust_per_cell_type = 3,
                              cluster_label = NULL, X_pca = NULL,
                              log_transformed = FALSE,
                              verbose = TRUE)

df_res[,'cell_type_org'] <- cell_type_test
head(df_res)

,cluster_label,cell_type_pred,cell_type_org
,<dbl>,<chr>,<fct>
Cy72_CD45_H02_S758_comb,15,B cell,B cell
CY58_1_CD45_B02_S974_comb,44,T cell,T cell
Cy71_CD45_D08_S524_comb,25,unassigned,Tumor
Cy81_FNA_CD45_B01_S301_comb,6,unassigned,Tumor
Cy80_II_CD45_B07_S883_comb,17,unassigned,Tumor
Cy81_Bulk_CD45_B10_S118_comb,6,unassigned,Tumor
